# 5일차 1교시 - Spark Repartition vs. Coalesce Explained
> 스파크는 분산 환경에서 병렬 처리를 잘하는 엔진인데 여기에서 병렬처리의 단위가 파티션이며 스파크 내부 구조에 의해 관리되지만 이용자에 의해 조정되기도 합니다.
> 그리고 스파크 어플리케이션을 수행하면서 가장 자주 많이 확인하게 되는 파티션 수에 대해 이해하고, 관련 이슈들을 어떻게 해결하는지 실습합니다.

### 목차
* 1. What is Partition ?
* 2. Why Partition needed ?
* 3. What is Replication ?
* 4. What is Coalesce ?
* 5. Which Repartition or Coalesce ?
* 6. References
  * [Spark Repartition & Coalesce](https://datanoon.com/blog/spark_repartition_coalesce)
  * [Spark Repartition vs. Coalesce](https://sparkbyexamples.com/spark/spark-repartition-vs-coalesce/)
  * https://medium.com/swlh/building-partitions-for-processing-data-files-in-apache-spark-2ca40209c9b7

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
spark

21/08/21 08:54:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 1. 'Partition'이란 무엇인가?
> 입력 데이터의 논리적인 청크들 혹은 데이터 덩어리라고 말할수 있으며, 스파크는 이러한 파티션을 물리적으로 다른 노드에 분산 저장한다
<br>

<!-- ![docker](image/docker.png)-->
<img src="images/docker.png" width="300" height="130" />

#### 아래와 같이 이러한 파티션 정보를 내부 Hash Partitioning Scheme 을 유지하는데 rdd.glom() 명령을 통해 확인할 수 있습니다

In [2]:
sc = spark.sparkContext
sc.parallelize(range(1,11)).getNumPartitions() == sc.defaultParallelism == 3

True

#### 1.1 파티션 정보를 어떻게 확인하는가?

In [3]:
# parallelize 함수를 통해 rdd 생성
rdd = sc.parallelize(range(1, 11))
rdd.getNumPartitions()

# RDD 의 경우 파티션 정보를 확인
rdd.glom().collect()

# Dataframe 의 경우 파티션 정보를 확인
df = rdd.map(lambda x: (x, )).toDF()
df.rdd.glom().collect()

[[Row(_1=1), Row(_1=2), Row(_1=3)],
 [Row(_1=4), Row(_1=5), Row(_1=6)],
 [Row(_1=7), Row(_1=8), Row(_1=9), Row(_1=10)]]

#### 1.2 파티션 수는 어떻게 결정나는가?
> 임의의 파일을 읽어서 데이터 프레임을 생성할 수도 있는데 이 때에 Hadoop2 기본 블록 사이즈가 128mb 이므로 약 1gb 데이터를 읽어들일 때에 약 10개의 파티션이 생성될 수 있으므로 파일이 128mb 보다 작은 경우는 1개의 파티션으로 읽어들일 것이다

### 2. 'Partition'은 왜 필요한가?
> 파일을 읽을 때부터 파티션을 결정할 수 있으며, 한 번에 수행할 수 있는 Executor 수를 결정짓기 때문에 병렬성을 결정 짓는 가장 큰 요소입니다.

In [4]:
numOfPartition = 1
rdd = sc.parallelize(range(10), numOfPartition)
rdd.collect() # Use rdd.glom().collect() instead

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

### 3. 'Replication' 무엇인가?
> 이미 생성된 파티션의 크기를 임의로 저장하는 명령이며 hash partitioned 된 데이터 파티션을 생성하며 모든 파티션은 동일한 크기를 가집니다.

#### 3.1 컬럼 지정을 통해 파티션을 구성

In [5]:
df = spark.range(1, 100)
ds = df.repartition("id") # 기본 키로 id 컬럼이 생성됩니다
ds.rdd.getNumPartitions()

5

> 위의 repartition 시에는 shuffle 이 발생하기 때문에 결과는 spark.sql.shuffle.partitions 값에 의해 파티션 수가 결정되므로 아래의 값을 확인합니다

In [6]:
spark.conf.get("spark.sql.shuffle.partitions")

'5'

#### 3.2 직접 파티션 수를 지정하는 경우

In [7]:
df.repartition(3).rdd.getNumPartitions()

3

### 4. 'Coalesce'는 무엇인가?
> 데이터프레임에 포함된 파티션의 수를 줄이는 명령입니다. 파티션 수는 절대 늘어날 수 없고 줄일 수만 있으며, 리파티션과 다르게 셔플이 발생하지 않으며, 각 파티션이 균등하게 분포된다는 보장은 없습니다. 

In [8]:
df = spark.range(1, 10)
df3 = df.repartition(3)
df3.rdd.getNumPartitions()

3

In [9]:
df3.coalesce(2).rdd.glom().collect() # 3개의 파티션이 2개로 줄어들기 때문에 2개가 1개로 merge 됩니다

[[Row(id=1), Row(id=4), Row(id=8)],
 [Row(id=3), Row(id=5), Row(id=9), Row(id=2), Row(id=6), Row(id=7)]]

In [10]:
df3.coalesce(8).rdd.glom().collect() # 오류는 나지 않으나 기존 파티션 수 3개가 그대로 유지됩니다

[[Row(id=1), Row(id=4), Row(id=8)],
 [Row(id=3), Row(id=5), Row(id=9)],
 [Row(id=2), Row(id=6), Row(id=7)]]

### 5. 'Repartition'과 'Coalesce' 중에서 언제 어떤 것을 쓰면 좋은가?

#### 5.1 Repartition
* 최종 파티션이 균등한 크기로 분포되기를 원할 때
* 파티션 수를 늘릴 필요가 있을 때
* 마지막 Stage 단계의 Reduce 작업이 충분히 큰 데이터 처리가 있어 병렬성을 보장 받아야 하지만, 최종 결과 데이터는 충분히 작은 파티션으로 생성되어야 하는 경우
![repartition](images/repartition.png)

#### 5.2 Coalesce
* 셔플을 발생시키지 않고 파티션 수를 줄이려고 할 때
* 파티션 수를 줄이기만 할 때
* 직전 Reduce 작업에 전달되는 데이터 크기가 충분히 작아서 Coalesce(#) 크기의 병렬성을 보장 받아도 충분히 빠른 경우
![coalesce](images/coalesce.png)

#### 5.3 비교 실습
> 각 천만 건의 레코드를 가진 데이터프레임의 Repartition 과 Coealesce 수행 시의 차이점을 확인하고 explain 을 통해 비교합니다
2개의 데이터 프레임을 생성하고, 1:1 조인이 되지 않도록 3의 배수, 2의 배수로 생성합니다.

In [11]:
df1 = spark.range(1, 3000000, 3) # 1:1 조인이 되지 않도록 3의 배수
df2 = spark.range(1, 2000000, 2) # 2의 배수로 숫자를 생성합니다
print(df1.rdd.getNumPartitions(), df2.rdd.getNumPartitions(), spark.conf.get("spark.sql.shuffle.partitions"))
df1.write.mode("overwrite").save("target/troubleshoot1/df1")
df2.write.mode("overwrite").save("target/troubleshoot1/df2")
df1.printSchema()

3 3 5


root
 |-- id: long (nullable = false)



> 기본적으로 생성되는 id 컬럼을 이용하여 inner join 을 수행하고, 각각 repartition 및 coalesce 를 수행합니다.

In [12]:
from pyspark.sql.functions import rand
df12 = df1.join(df2, "id")
# df12.show()
df12.write.mode("overwrite").save("target/troubleshoot1/df12")

finalNumOfPartition = 1
df3 = df12.repartition(finalNumOfPartition)
df3.write.mode("overwrite").save("target/troubleshoot1/df3")
df4 = df12.coalesce(finalNumOfPartition)
df4.write.mode("overwrite").save("target/troubleshoot1/df4")

In [13]:
def getNumPartitions(items):
    for item in items:
        print(item.getNumPartitions())

In [14]:
getNumPartitions([df1.rdd, df2.rdd, df12.rdd, df3.rdd, df4.rdd])

3
3
3
1
1


> 각 데이터 프레임의 파티션 수는 도커와 같은 컨테이너 환경에서 모든 리소스 매니저를 운영하기에 전체 코어 수에 바운드됩니다. 하지만 아래와 같이 파일로 저장한 이후에 다시 해당 파티션을 읽어오는 경우는 저장시의 파티션 수를 반드시 따르지는 않습니다.

In [15]:
rdd1 = spark.sparkContext.textFile("target/troubleshoot1/df1")
rdd2 = spark.sparkContext.textFile("target/troubleshoot1/df2")
rdd12 = spark.sparkContext.textFile("target/troubleshoot1/df12")
rdd3 = spark.sparkContext.textFile("target/troubleshoot1/df3")
rdd4 = spark.sparkContext.textFile("target/troubleshoot1/df4")

getNumPartitions([rdd1, rdd2, rdd12, rdd3, rdd4])

3
3
3
2
2


> 아래의 기본 설정에 따라 파티션을 읽고 쓸 때에 스파크 엔진에서 최적화를 수행합니다

In [16]:
defaultShufflePartitions = int(spark.conf.get("spark.sql.shuffle.partitions"))
defaultMinPartitions = sc.defaultMinPartitions
defaultParallelism = sc.defaultParallelism
defaultMaxPartitionBytes = int(spark.conf.get("spark.sql.files.maxPartitionBytes").replace("b", ""))
defaultOpenCostInBytes = int(spark.conf.get("spark.sql.files.openCostInBytes"))
out = """
defaultShufflePartitions: %d
defaultMinPartitions: %d
defaultParallelism: %d
maxPartitionBytes: %dmb
openCostInBytes: %dmb
""" % (defaultShufflePartitions, defaultMinPartitions, defaultParallelism, 
       defaultMaxPartitionBytes/(1024*1024), defaultOpenCostInBytes/(1024*1024))
print(out)


defaultShufflePartitions: 5
defaultMinPartitions: 2
defaultParallelism: 3
maxPartitionBytes: 128mb
openCostInBytes: 4mb



In [17]:
spark.sparkContext.getConf().getAll()

[('spark.sql.session.timeZone', 'Asia/Seoul'),
 ('spark.driver.host', '36066668d2b9'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.port', '41917'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.app.startTime', '1629536052159'),
 ('spark.rdd.compress', 'True'),
 ('spark.sql.warehouse.dir',
  'file:/home/jovyan/work/lgde-spark-troubleshoot/spark-warehouse'),
 ('spark.app.id', 'local-1629536053248'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.ui.showConsoleProgress', 'true')]

> 개별 데이터 프레임 Repartition 의 경우 이전 Join Stage 가 완료된 이후에 별도의 Stage 로 Exchange (shuffle) 후에 RoundRobinPartition 
http://localhost:4040/SQL/ 페이지를 통해 확인할 수 있으며, 조인 단계에서 200개의 병렬성을 그대로 활용할 수 있습니다.

![repartition](images/repartition.png)

In [18]:
df1.join(df2, "id").repartition(1).explain(True)

== Parsed Logical Plan ==
Repartition 1, true
+- Project [id#16L]
   +- Join Inner, (id#16L = id#18L)
      :- Range (1, 3000000, step=3, splits=Some(3))
      +- Range (1, 2000000, step=2, splits=Some(3))

== Analyzed Logical Plan ==
id: bigint
Repartition 1, true
+- Project [id#16L]
   +- Join Inner, (id#16L = id#18L)
      :- Range (1, 3000000, step=3, splits=Some(3))
      +- Range (1, 2000000, step=2, splits=Some(3))

== Optimized Logical Plan ==
Repartition 1, true
+- Project [id#16L]
   +- Join Inner, (id#16L = id#18L)
      :- Range (1, 3000000, step=3, splits=Some(3))
      +- Range (1, 2000000, step=2, splits=Some(3))

== Physical Plan ==
Exchange RoundRobinPartitioning(1), REPARTITION_WITH_NUM, [id=#317]
+- *(2) Project [id#16L]
   +- *(2) BroadcastHashJoin [id#16L], [id#18L], Inner, BuildRight, false
      :- *(2) Range (1, 3000000, step=3, splits=3)
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#312]
         +- *(1) Range

> 반면에 Coealesce 의 경우는 이전 Stage 인 Join 자체가 제한된 Reduce 작업을 통해 수행됩니다.
![coalesce](images/coalesce.png)

In [19]:
df1.join(df2, "id").coalesce(1).explain(True)

== Parsed Logical Plan ==
Repartition 1, false
+- Project [id#16L]
   +- Join Inner, (id#16L = id#18L)
      :- Range (1, 3000000, step=3, splits=Some(3))
      +- Range (1, 2000000, step=2, splits=Some(3))

== Analyzed Logical Plan ==
id: bigint
Repartition 1, false
+- Project [id#16L]
   +- Join Inner, (id#16L = id#18L)
      :- Range (1, 3000000, step=3, splits=Some(3))
      +- Range (1, 2000000, step=2, splits=Some(3))

== Optimized Logical Plan ==
Repartition 1, false
+- Project [id#16L]
   +- Join Inner, (id#16L = id#18L)
      :- Range (1, 3000000, step=3, splits=Some(3))
      +- Range (1, 2000000, step=2, splits=Some(3))

== Physical Plan ==
Coalesce 1
+- *(2) Project [id#16L]
   +- *(2) BroadcastHashJoin [id#16L], [id#18L], Inner, BuildRight, false
      :- *(2) Range (1, 3000000, step=3, splits=3)
      +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#353]
         +- *(1) Range (1, 2000000, step=2, splits=3)



> 실제 동일한 조인 연산을 수행할 때에 Repartition 과 Coalesce 가 성능에 어느정도 영향을 미치는지 확인해 봅니다.

In [20]:
%%time
df3.write.mode("overwrite").format("csv").save("target/troubleshoot1/repartition")

CPU times: user 1.6 ms, sys: 1.34 ms, total: 2.94 ms
Wall time: 957 ms


In [21]:
%%time
df4.write.mode("overwrite").format("csv").save("target/troubleshoot1/coalesce")

CPU times: user 397 µs, sys: 1.89 ms, total: 2.29 ms
Wall time: 627 ms
